시사뉴스 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait as ww
from bs4 import BeautifulSoup as bs
from collections import defaultdict
import time

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

base_url = "https://www.sisa-news.com/news/article_list_all.html?page={}"
article_base = "https://www.sisa-news.com/"

dict = defaultdict(list)
for page in range(1, 2):
    url = base_url.format(page)
    driver.get(url)
    time.sleep(2)

    # 기사 제목과 링크 추출
    
    ul_elem = driver.find_element(By.CSS_SELECTOR, 'ul.art_list_all')
    li_list = ul_elem.find_elements(By.TAG_NAME, 'li')


    for idx, li in enumerate(li_list):
        a_tags = li.find_elements(By.TAG_NAME, 'a')
        if not a_tags:
            continue

        href = a_tags[0].get_attribute('href')
        if href and 'no=' in href:
            parts = href.split('no=')
            if len(parts) > 1 and parts[1].isdigit():
                web_id = parts[1]
            
        article_url = 'https://www.sisa-news.com/news/article.html?no=' + web_id

        # open new window with article page
        driver.execute_script('window.open(arguments[0]);', article_url)
        driver.switch_to.window(driver.window_handles[1])

        article_soup = bs(driver.page_source, 'html.parser')

        dict['Category'].append(article_soup.select_one('#container > div.column.col73.mb00 > div:nth-child(1) > div > div.path_wrap > h3').get_text(strip=True))
        dict['Title'].append(article_soup.select_one('#container > div.column.col73.mb00 > div:nth-child(1) > div > div.arv_005_01 > div.fix_art_top > div > div > h2').get_text(strip=True))
        
        full_text = article_soup.select_one(
        '#container > div.column.col73.mb00 > div:nth-child(1) > div > div.arv_005_01 > div.fix_art_top > div > div > ul.art_info > li:nth-child(2)'
        ).get_text(strip=True)
        date_only = full_text.split()[1]

        dict['Date'].append(date_only)

        sentences = []

        atc = article_soup.select_one('#news_body_area')
        p_tags = atc.find_all('p')

        for p_tag in p_tags:
            text = p_tag.get_text(strip=True)
            trimmed = text[13:] if len(text) > 13 else ''
            sentences.append(trimmed)


        dict['Contents'].append(' '.join(sentences))


        driver.close()
        driver.switch_to.window(driver.window_handles[0])
driver.quit()

print(dict)

defaultdict(<class 'list'>, {'Category': ['바이오ㆍ제약', '전기ㆍ전자ㆍ통신ㆍ중화학', '사회', '자동차ㆍ교통', '문화', '자동차ㆍ교통', '바이오ㆍ제약', '문화', '지역네트워크', '지역네트워크', '지역네트워크', '지역네트워크', '지역네트워크'], 'Title': ['동아쏘시오그룹, 중대재해 대응 모의훈련 실시', "삼성전자, 중남미에서도 'AI 가전=삼성' 굳혀", '백석대 K-하이테크 플랫폼 사업단, 충남 디지털 전환 견인할 AI 드론 콘퍼런스 성료', 'KGM, 전기픽업 \'무쏘EV\' 호주 출시…"가격·상품성 승부"', '최고의 교육은 가정으로부터 시작된다', "현대모비스, 아이디어 대회 '2025 엠필즈 페스타' 개최", '품질의 삼성바이오로직스…"글로벌 제조승인 400건 돌파"', '누군가의 마음에 선물처럼 도착하는 시', '(사)한국여성농업인 영양군연합회 우리쌀 소비 촉진 홍보', '영양군종합자원봉사센터, 제13회 영양군 자원봉사대축제 개최', '금산군, 2026년 예산안 8,050억 원 편성', '정성이 담긴 김장김치로 따뜻한 겨울나기에 동참 대구북구새마을회, 2025 사랑의 김장 나눔 행사 실시', '대구 북구여성단체협의회, ‘2025년 사랑의 김장나눔’개최'], 'Date': ['2025.11.26', '2025.11.26', '2025.11.26', '2025.11.26', '2025.11.26', '2025.11.26', '2025.11.26', '2025.11.26', '2025.11.26', '2025.11.26', '2025.11.26', '2025.11.26', '2025.11.26'], 'Contents': [' 동아쏘시오홀딩스는 지난 25일 서울 동대문구 본사 O동 WINGS(윙스)에서 동아쏘시오그룹 임직원 및 협력사 근로자를 대상으로 중대재해 대응 모의훈련을 실시했다.  구성원의 응급 대응 역량을 강화하고 안전한 근무환경을 조성하기 위한 목적으로 마련됐다.  에 근무하고 있는 

In [2]:
import json

articles = []
for i in range(len(dict['Category'])):
    article = {
        "num": i+1,
        "Category": dict['Category'][i],
        "Date": dict['Date'][i],
        "Title": dict['Title'][i],
        "Contents": dict['Contents'][i]
    }
    articles.append(article)

# JSON으로 저장
with open('articles_sisa.json', 'w', encoding='utf-8') as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)
